In [1]:
import random

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

## Import des données

In [2]:
DATA_PATH = 'data/processed/'

In [3]:
interactions_df = pd.read_csv(DATA_PATH + 'interactions_df.csv', index_col=0)
interactions_df.head()

,user_id,article_id,click
0,3,233769,1
1,3,234686,1
2,3,235665,1
3,3,236065,1
4,3,236294,1


In [4]:
articles_min_df = pd.read_csv(DATA_PATH + 'articles_min.csv', index_col=0)
articles_min_df.head()


,category_id,created_at_ts,words_count,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,...,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249
article_id,,,,,,,,,,,,,,,,,,,,,
319488,433,2017-09-29 15:47:18,157,-0.466458,-0.964679,-0.485105,-0.877867,0.391511,0.483561,0.00548,...,-0.814141,-0.442375,0.039461,0.395648,-0.747964,-0.382282,-0.632840,-0.387871,-0.056827,-0.199358
360463,455,2017-10-01 10:01:03,176,-0.602885,-0.961464,-0.565603,-0.649518,-0.559716,0.524595,-0.37010,...,-0.585967,-0.653178,-0.195667,0.249913,0.331547,0.119053,0.659684,-0.244358,-0.151648,0.401354
360465,455,2017-10-01 13:13:54,114,-0.205855,-0.957574,-0.878344,-0.431890,-0.693044,0.168474,-0.38585,...,-0.557692,-0.653369,-0.120972,-0.227757,0.475501,0.446490,0.455878,-0.210448,-0.116186,0.585806
106520,228,2017-08-11 21:11:33,112,0.505829,-0.931578,-0.277545,0.383692,-0.603157,0.135672,-0.13724,...,0.267274,-0.060965,0.684367,-0.198938,-0.601258,0.362214,0.370468,0.369780,0.006491,0.607185
180261,301,2016-11-19 13:30:13,199,-0.631314,-0.972719,0.554279,0.087726,0.090802,0.394112,0.05757,...,0.300368,-0.433697,-0.092218,-0.694479,0.208018,-0.244026,0.054419,-0.172014,0.355977,-0.357948


## Split

In [5]:
interactions_train_df, interactions_test_df = train_test_split(interactions_df,
                                                               stratify=interactions_df['user_id'], 
                                                               test_size=0.20,
                                                               random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 17121
# interactions on Test set: 4281


### Création de rating

In [6]:
interactions_train_df

,user_id,article_id,click
7235,5832,284463,1
13152,11901,59758,1
16441,16610,332114,1
6313,4911,185992,1
9621,8192,207122,1
...,...,...,...
16585,16794,270589,1
6142,4761,207391,1
21172,27110,300299,1
6404,4972,42656,1


In [7]:
interactions_train_df[interactions_train_df[['user_id', 'article_id']].duplicated()]

,user_id,article_id,click


In [8]:
ratings_train_df = interactions_train_df.groupby(['user_id', 'article_id'])['article_id'].count().reset_index(name='click_count')

# calculer le nombre total de clics de chaque utilisateur
user_click_counts = ratings_train_df.groupby('user_id')['article_id'].count().reset_index(name='total_click_count')

# joindre les deux tables pour obtenir le nombre de clics de chaque utilisateur sur chaque article ainsi que le nombre total de clics de chaque utilisateur
ratings_train_df = ratings_train_df.merge(user_click_counts, on='user_id')

# calculer le rating de chaque article en utilisant le nombre de clics de l'utilisateur sur l'article normalisé par le nombre total de clics de l'utilisateur
ratings_train_df['rating'] = ratings_train_df['click_count'] / ratings_train_df['total_click_count']
ratings_train_df


,user_id,article_id,click_count,total_click_count,rating
0,3,233769,1,4,0.250000
1,3,234686,1,4,0.250000
2,3,235665,1,4,0.250000
3,3,236294,1,4,0.250000
4,5,59929,1,6,0.166667
...,...,...,...,...,...
17116,27876,293425,1,9,0.111111
17117,27903,160474,1,4,0.250000
17118,27903,161178,1,4,0.250000
17119,27903,207122,1,4,0.250000


In [9]:
ratings_train_df['total_click_count'].min()

2

In [10]:
ratings_train_df['rating'].min()

0.020833333333333332

# Popularity Recommander

In [11]:
popularity_df = pd.DataFrame({
    'article_id': interactions_train_df['article_id'].value_counts().index,
    'popularity': interactions_train_df['article_id'].value_counts().values,
})
popularity_df.head()

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
3,96663,617
4,119592,602


In [12]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['article_id'].isin(items_to_ignore)] \
                                                .sort_values('popularity', ascending= False)\
                                                .head(topn)
        return recommendations_df

In [13]:
popularity_model = PopularityRecommender(popularity_df, articles_min_df)

In [14]:
popularity_model.recommend_items(user_id=5890, items_to_ignore=set(interactions_test_df[interactions_test_df.user_id==5890].article_id), topn=5)

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
3,96663,617
5,59758,525


In [15]:
popularity_model.recommend_items(user_id=8192, items_to_ignore=set(interactions_test_df[interactions_test_df.user_id==8192].article_id), topn=5)

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
3,96663,617
4,119592,602


In [16]:
popularity_model.recommend_items(user_id=3, items_to_ignore=set(interactions_test_df[interactions_test_df.user_id==3].article_id), topn=5)

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
3,96663,617
4,119592,602


In [17]:
## Test avec filtre des articles déjà vus

In [18]:
set(interactions_test_df[interactions_test_df.user_id==8].article_id)

set()

In [19]:
popularity_model.recommend_items(user_id=8)

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
3,96663,617
4,119592,602
5,59758,525
6,336430,525
7,220466,326
8,313504,314
9,118180,280


In [20]:
set(interactions_test_df[interactions_test_df.user_id==9261].article_id)

{36690, 96663, 149734, 161585, 195586, 207035, 207391, 233605, 284664, 354701}

Il conseille un article déjà consulté :
 - 96 663 en 4e

In [21]:
popularity_model.recommend_items(user_id=9261, items_to_ignore = set(interactions_test_df[interactions_test_df.user_id==9261].article_id))

,article_id,popularity
0,284463,871
1,207122,729
2,160474,626
4,119592,602
5,59758,525
6,336430,525
7,220466,326
8,313504,314
9,118180,280
10,68866,276


# Content Based 

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, clicks, items_df):
        self.clicks = clicks
        self.items_df = items_df[['category_id', 'created_at_ts', 'words_count']]
        self.embeddings_df =  items_df.drop(columns=['category_id', 'created_at_ts', 'words_count'])

        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def get_user_profile(self, user_id):
        user_read_articles_id = list(set(self.clicks[self.clicks.user_id == user_id].article_id))
        user_profile = self.embeddings_df.loc[user_read_articles_id]
        return user_profile
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=5, verbose=False):
        user_profile = self.get_user_profile(user_id)
        cosine_similarities = cosine_similarity(user_profile, self.embeddings_df)
        max_similarities = cosine_similarities.max(axis=0)
        articles_similarities = pd.DataFrame({'article_id': self.embeddings_df.index, 'max_similarity': max_similarities})
        # Filtrer les articles déjà lus
        articles_similarities = articles_similarities[~articles_similarities['article_id'].isin(list(user_profile.index))]
        # Trier le DataFrame par ordre décroissant de similarité
        articles_similarities = articles_similarities.sort_values('max_similarity', ascending=False)
        return articles_similarities.head(topn)

In [24]:
content_based_recommender_model = ContentBasedRecommender(interactions_train_df, articles_min_df)

In [25]:
user_id = random.choice(list(set(interactions_train_df.user_id)))

In [26]:
content_based_recommender_model.recommend_items(user_id=user_id)

,article_id,max_similarity
556,108853,0.994266
898,118495,0.917994
896,118494,0.906185
552,108852,0.895575
1056,119231,0.891815


In [27]:
content_based_recommender_model.recommend_items(user_id=3)

,article_id,max_similarity
1678,236682,0.890063
1159,234427,0.881342
1666,236613,0.862717
1382,235373,0.860739
1244,234798,0.859561


In [28]:
for k in [233769, 235665, 234686, 236294]:
    print(k in articles_min_df.index)

True
True
True
True


In [29]:
articles_min_df.index.dtype

dtype('int64')

### Méthode sur un user

self.clicks sera interactions_train_df

self.items_df :

In [30]:
items_df = articles_min_df[['category_id', 'created_at_ts', 'words_count']]

self.embedding_df :

In [31]:
embedding_min_df = articles_min_df.drop(columns=['category_id', 'created_at_ts', 'words_count'])
embedding_min_df

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249
article_id,,,,,,,,,,,,,,,,,,,,,
319488,-0.466458,-0.964679,-0.485105,-0.877867,0.391511,0.483561,0.005480,-0.211324,-0.020051,-0.174135,...,-0.814141,-0.442375,0.039461,0.395648,-0.747964,-0.382282,-0.632840,-0.387871,-0.056827,-0.199358
360463,-0.602885,-0.961464,-0.565603,-0.649518,-0.559716,0.524595,-0.370100,0.361097,-0.108726,0.181928,...,-0.585967,-0.653178,-0.195667,0.249913,0.331547,0.119053,0.659684,-0.244358,-0.151648,0.401354
360465,-0.205855,-0.957574,-0.878344,-0.431890,-0.693044,0.168474,-0.385850,0.432571,-0.696537,0.027664,...,-0.557692,-0.653369,-0.120972,-0.227757,0.475501,0.446490,0.455878,-0.210448,-0.116186,0.585806
106520,0.505829,-0.931578,-0.277545,0.383692,-0.603157,0.135672,-0.137240,0.333256,0.247378,0.298668,...,0.267274,-0.060965,0.684367,-0.198938,-0.601258,0.362214,0.370468,0.369780,0.006491,0.607185
180261,-0.631314,-0.972719,0.554279,0.087726,0.090802,0.394112,0.057570,-0.612913,0.182290,-0.118985,...,0.300368,-0.433697,-0.092218,-0.694479,0.208018,-0.244026,0.054419,-0.172014,0.355977,-0.357948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32730,-0.063069,-0.963189,-0.020831,0.328858,0.314361,-0.227227,0.389601,0.099065,0.672383,0.442416,...,0.528567,0.427474,0.461421,-0.461597,-0.333884,0.534221,-0.199299,-0.593895,0.302743,-0.524815
352227,-0.037319,-0.970653,0.235284,-0.792944,-0.652155,-0.429864,0.559105,-0.071258,0.550386,-0.846791,...,-0.535248,-0.867873,-0.768359,0.483744,0.029526,0.570641,0.170771,-0.608991,0.154890,0.234986
311282,-0.320332,-0.972268,0.788268,-0.835453,-0.814294,0.650294,0.580025,-0.756977,0.738576,-0.058501,...,-0.786265,-0.272901,-0.877875,0.703472,-0.748671,0.258157,-0.033027,0.110924,-0.687876,-0.688037


###  Article consulté par un utilisateur

#### Méthode : get_user_profile

In [32]:
interactions_train_df

,user_id,article_id,click
7235,5832,284463,1
13152,11901,59758,1
16441,16610,332114,1
6313,4911,185992,1
9621,8192,207122,1
...,...,...,...
16585,16794,270589,1
6142,4761,207391,1
21172,27110,300299,1
6404,4972,42656,1


In [33]:
user_id = 3

In [34]:
user_read_articles_id = set(interactions_train_df[interactions_train_df.user_id == user_id].article_id)
user_profile = embedding_min_df.loc[user_read_articles_id]
user_profile

C:\Users\USER\AppData\Local\Temp\ipykernel_2472\2042029245.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  user_profile = embedding_min_df.loc[user_read_articles_id]


,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249
article_id,,,,,,,,,,,,,,,,,,,,,
233769,-0.828736,-0.974723,-0.234350,-0.507602,-0.203012,-0.755248,-0.788898,0.013411,-0.338640,-0.556146,...,-0.570091,-0.148384,0.479919,-0.910203,-0.529137,0.142069,-0.476662,0.485083,0.479215,0.014373
235665,-0.513240,-0.963361,-0.602106,-0.206739,-0.412994,-0.115276,-0.732224,-0.266477,0.507747,-0.671837,...,-0.205707,0.124613,0.267336,-0.428892,0.081707,0.299926,0.351382,0.871408,0.290365,0.440695
234686,-0.538448,-0.948071,-0.705048,-0.083332,-0.151408,-0.359961,-0.562148,-0.086911,-0.066602,0.117681,...,0.193455,0.193878,0.146542,-0.761525,0.200309,-0.242208,-0.492294,-0.285201,0.278869,0.238224
236294,-0.791363,-0.967025,-0.247181,-0.462593,0.238766,-0.713218,-0.789062,-0.026739,-0.222805,-0.163980,...,-0.609091,0.230817,0.090354,-0.457567,-0.026513,-0.433131,0.347777,0.511057,0.470070,0.092472


#### Méthode : recommend_items

In [35]:
cosine_similarities = cosine_similarity(user_profile, embedding_min_df)
cosine_similarities.shape

(4, 1842)

In [36]:
max_similarities = cosine_similarities.max(axis=0)
len(max_similarities)

1842

In [37]:
articles_similarities = pd.DataFrame({'article_id': embedding_min_df.index, 'max_similarity': max_similarities})
# Filtrer les articles déjà lus
articles_similarities = articles_similarities[~articles_similarities['article_id'].isin(list(user_profile.index))]
# Trier le DataFrame par ordre décroissant de similarité
articles_similarities = articles_similarities.sort_values('max_similarity', ascending=False)

# Sélectionner les 5 articles avec les valeurs maximales de similarité
top_5_articles = articles_similarities.head(5)

# Afficher les articles recommandés avec leurs valeurs maximales de similarité
print(top_5_articles)

      article_id  max_similarity
1678      236682        0.890063
1159      234427        0.881342
1666      236613        0.862717
1382      235373        0.860739
1244      234798        0.859561


In [38]:
user_clicks_df = interactions_train_df[interactions_train_df['user_id']==user_id]

In [39]:
user_clicks_df['article_id']

0    233769
4    236294
2    235665
1    234686
Name: article_id, dtype: int64

### Divers tests

In [40]:
content_based_recommender_model = ContentBasedRecommender(interactions_train_df, articles_min_df)

In [41]:
content_based_recommender_model.recommend_items(user_id=user_id)

,article_id,max_similarity
1678,236682,0.890063
1159,234427,0.881342
1666,236613,0.862717
1382,235373,0.860739
1244,234798,0.859561


# Collaborative Filtering Method

In [42]:
interactions_train_df

,user_id,article_id,click
7235,5832,284463,1
13152,11901,59758,1
16441,16610,332114,1
6313,4911,185992,1
9621,8192,207122,1
...,...,...,...
16585,16794,270589,1
6142,4761,207391,1
21172,27110,300299,1
6404,4972,42656,1


In [43]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='user_id', 
                                                          columns='article_id', 
                                                          values='click').fillna(0)

users_items_pivot_matrix_df.head(10)

article_id,1916,1933,2136,2137,2288,2662,3147,3201,3330,3515,...,362928,363018,363033,363127,363291,363330,363633,363925,363976,364001
user_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[3, 5, 16, 18, 22, 24, 32, 44, 51, 53]

In [46]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

<3169x1606 sparse matrix of type '<class 'numpy.float64'>'
	with 17121 stored elements in Compressed Sparse Row format>

In [47]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [48]:
U.shape

(3169, 15)

In [49]:
Vt.shape

(15, 1606)

In [50]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [51]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-1.07137647e-06, -1.26518221e-04,  3.01622940e-04, ...,
         4.66118953e-06,  2.44957682e-04, -1.14766699e-04],
       [ 3.90018265e-06, -7.60817691e-04,  1.24285734e-04, ...,
        -4.16953324e-04,  2.96125890e-03,  3.26052735e-03],
       [ 1.87015346e-06, -1.67197432e-03, -3.33149248e-04, ...,
         5.74113628e-04,  6.59300834e-04,  6.07269345e-04],
       ...,
       [ 5.23789327e-06,  1.38447976e-04, -2.42628411e-06, ...,
        -1.49759020e-04,  6.14390464e-04,  1.03054598e-03],
       [ 3.83166267e-06, -1.88168832e-03,  1.49453823e-03, ...,
         6.88540870e-04,  6.34008024e-03,  8.49227031e-04],
       [ 7.34966857e-06,  6.46750626e-04, -7.02211257e-04, ...,
        -5.55429329e-04, -7.99177658e-04,  3.38833894e-03]])

In [52]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())


In [53]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns=users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,3,5,16,18,22,24,32,44,51,53,...,27747,27750,27756,27765,27766,27792,27813,27849,27876,27903
article_id,,,,,,,,,,,,,,,,,,,,,
1916,0.098585,0.098587,0.098586,0.098586,0.098587,0.098589,0.098586,0.098584,0.098582,0.098582,...,0.098588,0.098586,0.098586,0.098585,0.098589,0.098587,0.098588,0.098587,0.098587,0.098587
1933,0.098556,0.098410,0.098200,0.098561,0.098638,0.098365,0.098489,0.098504,0.098707,0.098291,...,0.098783,0.098586,0.098789,0.098219,0.099113,0.098293,0.098538,0.098618,0.098152,0.098735
2136,0.098655,0.098614,0.098509,0.098591,0.098753,0.098486,0.099321,0.098640,0.099668,0.099104,...,0.098545,0.098586,0.099135,0.098984,0.098857,0.098446,0.098576,0.098585,0.098930,0.098424
2137,0.098572,0.098666,0.098659,0.098570,0.098673,0.098684,0.098602,0.098551,0.099090,0.098357,...,0.098951,0.098586,0.098043,0.098360,0.098140,0.098464,0.098359,0.098468,0.099174,0.098552
2288,0.098595,0.098682,0.098616,0.098597,0.098676,0.098668,0.098584,0.098581,0.098617,0.098627,...,0.098630,0.098586,0.098598,0.098652,0.098659,0.098624,0.098656,0.098636,0.098601,0.098609
2662,0.098560,0.098753,0.099541,0.098557,0.098376,0.099052,0.098209,0.098443,0.098031,0.098324,...,0.098463,0.098586,0.099037,0.099197,0.098253,0.099421,0.098780,0.098708,0.098401,0.098871
3147,0.098764,0.098184,0.098461,0.098607,0.098734,0.097741,0.098335,0.098822,0.100369,0.099850,...,0.098341,0.098586,0.100479,0.100362,0.097733,0.099034,0.097891,0.098213,0.098181,0.097842
3201,0.098560,0.098753,0.099541,0.098557,0.098376,0.099052,0.098209,0.098443,0.098031,0.098324,...,0.098463,0.098586,0.099037,0.099197,0.098253,0.099421,0.098780,0.098708,0.098401,0.098871
3330,0.098586,0.098551,0.098533,0.098585,0.098851,0.098513,0.098556,0.098444,0.098475,0.098375,...,0.098552,0.098586,0.098623,0.098585,0.098561,0.098586,0.098546,0.098551,0.098859,0.098504


In [54]:
len(cf_preds_df.columns)

3169

In [55]:
sorted_user_predictions = cf_preds_df[3].sort_values(ascending=False).reset_index().rename(columns={3: 'recStrength'})

In [56]:
items_to_ignore = []

In [57]:
sorted_user_predictions[~sorted_user_predictions['article_id'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(5)

,article_id,recStrength
0,233605,0.114615
1,235210,0.109861
2,284847,0.109818
3,108854,0.103631
4,235840,0.103030


In [58]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['article_id'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'article_id', 
                                                          right_on = 'article_id')[['recStrength', 'article_id']]


        return recommendations_df
    


In [59]:
cf_recommender_model = CFRecommender(cf_preds_df, articles_min_df)

In [60]:
cf_recommender_model.recommend_items(user_id=3)

,article_id,recStrength
0,233605,0.114615
1,235210,0.109861
2,284847,0.109818
3,108854,0.103631
4,235840,0.103030
5,235263,0.103013
6,235132,0.102428
7,195586,0.101955
8,157132,0.101055
9,96663,0.101053


# CF Avec Surprise 

In [61]:
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate

ModuleNotFoundError: No module named 'surprise'

In [ ]:
ratings_train_df = ratings_train_df[['user_id', 'article_id', 'rating']].rename(columns={"user_id": 'itemID', 'article_id': "userID"})
ratings_train_df

In [ ]:
ratings_train_df

## Evaluation

In [129]:
# import random
# import numpy as np

# class TopNAccuracyEvaluator:
    
#     def __init__(self, test_data, topn=5):
#         self.test_data = test_data
#         self.topn = topn
        
#     def evaluate_model(self, model):
#         total_hits = 0
#         total_users = len(self.test_data['user_id'].unique())
        
#         for user_id in self.test_data['user_id'].unique():
#             interacted_items = self.test_data[self.test_data['user_id'] == user_id]['article_id'].values
#             non_interacted_items = self.get_non_interacted_items(user_id)
            
#             items_to_rank = np.append(interacted_items, non_interacted_items)
#             scores = model.rank_items(user_id, items_to_rank)
            
#             ranked_items = [item for _, item in sorted(zip(scores, items_to_rank), reverse=True)]
#             topn_items = ranked_items[:self.topn]
            
#             hits = len(set(topn_items) & set(interacted_items))
#             total_hits += hits
        
#         topn_accuracy = total_hits / (total_users * self.topn)
#         return topn_accuracy
    
#     def get_non_interacted_items(self, user_id):
#         interacted_items = self.test_data[self.test_data['user_id'] == user_id]['article_id'].values
#         all_items = set(self.test_data['article_id'].unique())
#         non_interacted_items = list(all_items - set(interacted_items))
#         return random.sample(non_interacted_items, 100)

In [130]:
# # Créer une instance de l'évaluateur avec les données de test et le nombre de recommandations à évaluer (par exemple, top 5)
# evaluator = TopNAccuracyEvaluator(test_data, topn=5)

# # Évaluer un modèle de recommandation spécifique (par exemple, content-based model)
# accuracy = evaluator.evaluate_model(content_based_model)

# # Afficher la précision Top-N obtenue
# print(f"Top-{evaluator.topn} Accuracy: {accuracy}")

### Evaluateur

In [64]:
def get_articles_interacted(user_id, interactions_df):
    return set(interactions_df[interactions_df.user_id == user_id].article_id)

In [65]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, user_id, sample_size, seed=42):
        interacted_items = get_articles_interacted(user_id, interactions_df)                                   ######### ???????????????
        all_items = set(interactions_df['article_id'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, user_id):
        
        #Getting the items in test set
        interacted_values_testset = interactions_test_df[interactions_test_df.user_id == user_id]
        if type(interacted_values_testset['article_id']) == pd.Series:
            user_interacted_items_testset = set(interacted_values_testset['article_id'])
        else:
            user_interacted_items_testset = set([int(interacted_values_testset['article_id'])])  
        user_interacted_items_count_testset = len(user_interacted_items_testset) 
        

        #Getting a ranked recommendation list from a model for a given user
        user_recs_df = model.recommend_items(user_id, 
                                             items_to_ignore=get_articles_interacted(user_id, interactions_train_df), 
                                             topn=10000000000)
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        # print(user_interacted_items_testset)
        #For each item the user has interacted in test set
        for item_id in user_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(user_id, 
                                                                               sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                               seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = user_recs_df[user_recs_df['article_id'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['article_id'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(user_interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(user_interacted_items_count_testset)

        person_metrics = {'hits@5_count': hits_at_5_count, 
                          'hits@10_count': hits_at_10_count, 
                          'interacted_count': user_interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users
        people_metrics = []
        for idx, user_id in enumerate(list(interactions_test_df['user_id'].unique())):
            if idx % 100 == 0 and idx > 0:
                print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, user_id)  
            person_metrics['_user_id'] = user_id
            people_metrics.append(person_metrics)
        # print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [66]:
# model_evaluator.evaluate_model_for_user(popularity_model, 17745)  

### Modèles

In [67]:
popularity_model = PopularityRecommender(popularity_df, articles_min_df)

In [68]:
%%time
print('Evaluating Popularity recommendation model... \n')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)

Evaluating Popularity recommendation model... 



C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)


100 users processed
200 users processed
300 users processed
400 users processed
500 users processed
600 users processed
700 users processed
800 users processed
900 users processed
1000 users processed
1100 users processed
1200 users processed
1300 users processed
1400 users processed
1500 users processed
1600 users processed
1700 users processed
1800 users processed
1900 users processed
2000 users processed
2100 users processed
2200 users processed
2300 users processed
2400 users processed
2500 users processed
2600 users processed
2700 users processed
2800 users processed
2900 users processed
3000 users processed
3100 users processed
CPU times: total: 11.9 s
Wall time: 11.9 s


In [69]:
print('\nGlobal metrics:\n%s' % pop_global_metrics)


Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.7502919878533053, 'recall@10': 0.8325157673440785}


In [70]:
pop_detailed_results_df.head(10)

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_user_id
346,3,5,12,0.250000,0.416667,5890
61,7,7,10,0.700000,0.700000,9261
265,0,0,9,0.000000,0.000000,15275
36,4,5,7,0.571429,0.714286,681
185,5,5,7,0.714286,0.714286,4568
810,3,5,7,0.428571,0.714286,1819
161,2,3,6,0.333333,0.500000,169
51,5,6,6,0.833333,1.000000,2930
494,5,5,6,0.833333,0.833333,6753
44,4,4,5,0.800000,0.800000,19320


### Content Based

In [71]:
content_based_recommender_model = ContentBasedRecommender(interactions_train_df, articles_min_df)

In [72]:
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)

C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

100 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

200 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

...

C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

3000 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

3100 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

In [73]:
print('\nGlobal metrics:\n%s' % pop_global_metrics)


Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.20953048353188508, 'recall@10': 0.30390095772015885}


In [74]:
pop_detailed_results_df.head(10)

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_user_id
346,1,2,12,0.083333,0.166667,5890
61,2,3,10,0.200000,0.300000,9261
265,8,9,9,0.888889,1.000000,15275
36,1,1,7,0.142857,0.142857,681
185,3,3,7,0.428571,0.428571,4568
810,0,0,7,0.000000,0.000000,1819
161,0,1,6,0.000000,0.166667,169
51,0,1,6,0.000000,0.166667,2930
494,2,2,6,0.333333,0.333333,6753
44,1,1,5,0.200000,0.200000,19320


## Collaborative filtering

In [75]:
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

NameError: name 'articles_df' is not defined

In [76]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

100 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

200 users processed


...

C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

3000 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

3100 users processed


C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\USER\AppData\Local\Temp\ipykernel_2472\3129041406.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sa

In [ ]:
print('\nGlobal metrics:\n%s' % cf_global_metrics)

In [ ]:
cf_detailed_results_df.head(10)

In [ ]:
interactions_train_df.to_csv('interactions_df_e.csv')
interactions_train_df.head()

In [ ]:
articles_min_df.to_csv('articles_df_e.csv')
articles_min_df.head()

# Export

In [ ]:
articles_min_df

In [ ]:
articles_min_df

In [ ]:
cf_recommender_model.recommend_items(user_id=3, topn=5)

In [ ]:
popularity_model.recommend_items(user_id=3, topn=5)

# Brouillon

### Tests évaluateurs

#### Popularity

In [ ]:
pop_detailed_results_df[pop_detailed_results_df._user_id == 17745]

In [ ]:
model_evaluator.evaluate_model_for_user(popularity_model, 17745)  

In [ ]:
interactions_train_df[interactions_train_df.user_id == 17745]

In [ ]:
get_articles_interacted(17745, interactions_train_df)

1.

In [ ]:
interacted_values_testset = interactions_test_df[interactions_test_df.user_id == 17745]
interacted_values_testset

In [ ]:
user_interacted_items_testset = set(interacted_values_testset['article_id'])
user_interacted_items_testset

In [ ]:
user_interacted_items_count_testset = len(user_interacted_items_testset) 
user_interacted_items_count_testset

In [ ]:
#Getting a ranked recommendation list from a model for a given user
user_recs_df = popularity_model.recommend_items(17745,
                                                items_to_ignore=get_articles_interacted(17745, interactions_train_df),
                                                topn=10000000000)
user_recs_df

In [ ]:
user_interacted_items_testset

In [ ]:
def get_not_interacted_items_sample(user_id, sample_size, seed=42):
    interacted_items = get_articles_interacted(user_id, interactions_df)
    
    all_items = set(interactions_df['article_id'])
    non_interacted_items = all_items - interacted_items
    random.seed(seed)
    non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
    return set(non_interacted_items_sample)

In [ ]:
get_articles_interacted(17745, interactions_df)

pour 59758

In [ ]:
#Getting a random sample (100) items the user has not interacted 
#(to represent items that are assumed to be no relevant to the user)
seed = 59758 % (2**32)
print(seed)
non_interacted_items_sample = get_not_interacted_items_sample(17745,
                                                              sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                              seed=seed)

In [ ]:
#Combining the current interacted item with the 100 random items
items_to_filter_recs = non_interacted_items_sample.union(set([59758]))
len(items_to_filter_recs)

In [ ]:
user_recs_df

In [ ]:
user_recs_df['article_id'].isin(items_to_filter_recs).sum()

In [ ]:
user_recs_df[user_recs_df['article_id'].isin(items_to_filter_recs)]

In [ ]:
#Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
valid_recs_df = user_recs_df[user_recs_df['article_id'].isin(items_to_filter_recs)]
valid_recs_df

In [ ]:
user_recs_df['article_id'].isin(items_to_filter_recs).sum()

In [ ]:
valid_recs = valid_recs_df['article_id'].values
valid_recs

In [ ]:
59758 in valid_recs

In [ ]:
ModelEvaluator()._verify_hit_top_n(59758, valid_recs, 5)

272660

In [ ]:
#Getting a random sample (100) items the user has not interacted 
#(to represent items that are assumed to be no relevant to the user)
seed = 272660 % (2**32)
print(seed)
non_interacted_items_sample = get_not_interacted_items_sample(17745,
                                                              sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                              seed=seed)

In [ ]:
#Combining the current interacted item with the 100 random items
items_to_filter_recs = non_interacted_items_sample.union(set([272660]))
len(items_to_filter_recs)

In [ ]:
ModelEvaluator()._verify_hit_top_n(272660, valid_recs, 10)

## Brouillon

In [ ]:
def get5Articles(article_emb, userId, frame):
    #get all articles read by user
    var= frame.loc[frame['user_id']==userId]['click_article_id'].tolist()
    #we choose the last element of the list
    value = var[-1]
    print(value)

    #get 5 articles the most similar to the selected one
    distances = distance.cdist([article_emb[value]],article_emb, "cosine")[0]
    #find indexes except the one selected
    result = np.argsort(distances)[1:6]
    #similarity value between selected article and 5 top proposed articles (the smaller the better!)
    similarite = distance.cdist([article_emb[value]],article_emb[result], "cosine")[0]
    print(result)
    print(similarite)

    return result, similarite
var= dataframe.loc[dataframe['user_id']==userId]['click_article_id'].tolist()
print(len(var))

In [ ]:
from pandas.api.types import is_numeric_dtype
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import StandardScaler


def aggregate_articles(articles):
    return articles.groupby(lambda x: True).agg(
        {
            col: "mean"
            if is_numeric_dtype(articles.dtypes[col])
            else lambda x: x.mode()[0]
            for col in articles.columns
        }
    )


def get_user_interest(user_id, clicks, articles, strategy="last_click"):
    user_id = str(user_id)

    if strategy == "last_click":
        last_clicked_article_id = (
            clicks.query("user_id == @user_id")
            .sort_values("click_timestamp", ascending=False)
            .reset_index(drop=True)
            .iloc[0]["click_article_id"]
        )
        interest = articles.query("article_id == @last_clicked_article_id")

    elif strategy == "last_session":
        last_session_id = (
            clicks.query("user_id == @user_id")
            .sort_values("click_timestamp", ascending=False)
            .reset_index(drop=True)
            .iloc[0]["session_id"]
        )
        session_article_ids = clicks.query("session_id == @last_session_id")[
            "click_article_id"
        ]
        interest = aggregate_articles(
            articles.query("article_id in @session_article_ids")
        ).drop(["article_id"], axis=1)

    elif strategy == "all_clicks":
        all_article_ids = clicks.query("user_id == @user_id")["click_article_id"]
        interest = aggregate_articles(
            articles.query("article_id in @all_article_ids")
        ).drop(["article_id"], axis=1)

    else:
        raise NotImplementedError

    return interest


def prepare_for_scale(articles, category_id):
    articles_copy = articles.drop(["article_id", "similarity"], axis=1, errors="ignore")
    articles_copy["category_id"] = articles_copy["category_id"].apply(
        lambda x: category_id if int(x) == category_id else 0
    )
    articles_copy["created_at_ts"] = articles_copy["created_at_ts"].apply(
        lambda x: x.value
    )

    return articles_copy


def get_closest_articles(interest, articles, n=10):
    category_id = interest["category_id"].iloc[0]

    scaler = StandardScaler()
    articles_std = scaler.fit_transform(prepare_for_scale(articles, category_id))
    interest_std = scaler.transform(prepare_for_scale(interest, category_id))

    articles = articles.copy()
    articles["similarity"] = cosine_similarity(interest_std, articles_std)[0]

    return (
        articles.sort_values("similarity", ascending=False).iloc[:n],
        scaler,
        articles_std,
        interest_std,
    )

In [ ]:
user_id = "5891"

all_article_ids = clicks.query("user_id == @user_id")["click_article_id"]
all_article_ids


In [ ]:
articles.query("article_id in @all_article_ids")

In [ ]:
interest = aggregate_articles(
    articles.query("article_id in @all_article_ids")
).drop(["article_id"], axis=1)

In [ ]:
interest = get_user_interest(user_id, clicks, articles, strategy="all_clicks")
interest

In [ ]:
category_id = interest["category_id"].iloc[0]

closest_articles, scaler, articles_std, interest_std = get_closest_articles(
    interest, articles
)

articles_sample_std = scaler.transform(prepare_for_scale(articles_sample, category_id))
closest_articles_std = scaler.transform(
    prepare_for_scale(closest_articles, category_id)
)


closest_articles.head()

In [ ]:
def get_item_profile(item_id, embedding):
    idx = item_ids.index(item_id)
    item_profile = embedding[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
                                                   .isin(articles_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [ ]:
def get_items_profiles(ids, embeddings_df):
    item_profiles = embeddings_df.loc[ids]
    return item_profiles

In [ ]:
idx=1
embeddings_df[idx:idx+1]

In [ ]:

scaler = MinMaxScaler()
articles_metadata['words_count_normalized'] = scaler.fit_transform(articles_metadata[['words_count']])
metadata_embeddings = pd.concat([articles_metadata, pd.DataFrame(embeddings)], axis=1)

In [ ]:


# Etape 3 : Calcul de la similarité cosinus
similarities = cosine_similarity(embeddings)

In [ ]:
# Etape 4 : Recommandation
def get_top_N_similar_articles(article_id, N):
    # Obtenir les indices de similarité les plus élevés pour l'article donné
    similar_indices = np.argsort(similarities[article_id])[::-1]
    # Exclure l'article donné lui-même de la liste des articles similaires
    similar_indices = [i for i in similar_indices if i != article_id]
    # Retourner les N articles les plus similaires
    return similar_indices[:N]

In [ ]:
def get_item_profile(item_id, embeddings_df):
    if item_id in embeddings_df.index:
        item_profile = embeddings_df.loc[item_id]
        return item_profile
    else:
        raise Exception(f"Item ID {item_id} not found in embeddings dataframe.")

In [ ]:
def test_get_item_profile_existing_item():
    # Préparer les données de test
    item_id = 233769
    item_embedding = np.array([-0.8287356 , -0.97472274, -0.2343504 , -0.507602  , -0.20301214,
       -0.7552476 , -0.7888979 ,  0.01341149, -0.33863962, -0.5561465 ,
        0.3100278 , -0.778982  ,  0.7512819 , -0.23324044,  0.4121713 ,
       -0.5426781 ,  0.47454038, -0.22696275, -0.10203543, -0.5024353 ,
       -0.60398304, -0.54930454,  0.34155002,  0.12378635,  0.8087954 ,
        0.10313021, -0.5370047 ,  0.4849902 , -0.4675273 ,  0.34045044,
        0.5480398 ,  0.74296916,  0.59673303, -0.16853915, -0.3799169 ,
       -0.56623673,  0.6041001 ,  0.64344656, -0.10865439, -0.52409196,
        0.57892364, -0.77013475, -0.08198519, -0.9627955 , -0.71177995,
       -0.14029205,  0.11892216, -0.7764405 ,  0.7397275 , -0.42856127,
       -0.33112493, -0.15072216,  0.14735089,  0.10270992,  0.5683811 ,
        0.45626995, -0.0612862 ,  0.3034936 ,  0.9501255 ,  0.04021438,
        0.9164283 ,  0.00381889, -0.6349429 ,  0.33859843, -0.7370531 ,
       -0.31408015,  0.15340929, -0.05552195, -0.2779824 , -0.02695254,
       -0.17171979,  0.51752853, -0.5474673 ,  0.8174122 ,  0.36185524,
       -0.42525604, -0.60325485, -0.9502143 , -0.52277863, -0.30835652,
        0.2005286 , -0.33828133,  0.3936592 , -0.48687726, -0.48575693,
       -0.85266876,  0.6769977 ,  0.7080853 ,  0.05744236, -0.7740926 ,
        0.13894856, -0.05714169,  0.1355859 ,  0.7953611 ,  0.25262952,
        0.0941165 ,  0.44681984,  0.23953067, -0.13841373,  0.02878353,
        0.67391896, -0.44141293, -0.5531681 ,  0.16828513, -0.12672912,
        0.0297437 , -0.16047087, -0.274442  , -0.06182434,  0.38594875,
        0.02526377,  0.7747233 , -0.63108146,  0.05201101, -0.32181782,
       -0.32501397,  0.5105953 , -0.74021864, -0.64390665,  0.4577492 ,
       -0.7038889 , -0.06114122, -0.5547145 ,  0.57443976,  0.3123108 ,
       -0.17405212, -0.5812771 ,  0.90447813,  0.07316294, -0.18762887,
       -0.7980098 , -0.32147646, -0.4024939 , -0.07141314,  0.9660965 ,
        0.43441543, -0.70371723,  0.61786413, -0.0758319 , -0.28787193,
       -0.8399298 , -0.43915266, -0.20963988,  0.71566814, -0.22547337,
        0.24714744,  0.5595125 ,  0.25358436, -0.53577185, -0.34129727,
        0.9355614 ,  0.65643483, -0.7987242 ,  0.8178398 ,  0.22921982,
        0.6823472 ,  0.7870077 ,  0.5701958 , -0.28496158,  0.1632245 ,
        0.8934072 , -0.23540604,  0.6992183 ,  0.12849793,  0.6064123 ,
        0.07259716,  0.49668622, -0.46898538,  0.19841053, -0.07045401,
       -0.8902309 , -0.6055508 ,  0.7803485 ,  0.7901738 , -0.3727045 ,
        0.486982  , -0.1812807 ,  0.30095914, -0.34488356,  0.202357  ,
       -0.3516529 , -0.28802493,  0.08888996,  0.67288804,  0.5793077 ,
        0.9206487 ,  0.7301116 ,  0.08399629,  0.42067733,  0.3177738 ,
       -0.63564265,  0.25298655, -0.64491785, -0.04334317,  0.38038555,
       -0.29716125,  0.8778306 , -0.19676545, -0.966003  , -0.17732191,
        0.2200703 ,  0.28344923, -0.26868   , -0.18584733, -0.71617645,
       -0.55341566, -0.27159512,  0.07829893, -0.8633069 ,  0.63711536,
       -0.876505  , -0.05230286,  0.2272165 , -0.42809567, -0.8010121 ,
       -0.1288896 , -0.13892251, -0.69238293, -0.6063701 ,  0.60368   ,
        0.53202295,  0.9496571 , -0.0093426 ,  0.8527407 ,  0.3178616 ,
        0.5168774 , -0.75331414,  0.03515351, -0.5380031 , -0.36240333,
        0.5035036 ,  0.5317434 , -0.65624094, -0.46192008, -0.42386025,
       -0.85975856,  0.8018809 , -0.7201011 ,  0.65319014,  0.26274657,
       -0.57009065, -0.14838396,  0.47991896, -0.9102029 , -0.52913684,
        0.14206943, -0.47666222,  0.48508337,  0.47921517,  0.0143735 ],
      dtype='float32')
    embeddings_df = pd.DataFrame([item_embedding], index=[item_id])

    # Appeler la méthode à tester
    item_profile = get_item_profile(item_id, embeddings_df)

    # Vérifier si le profil renvoyé est correct
    assert np.array_equal(item_profile, np.array(item_embedding)), "Failed: Incorrect item profile returned."

def test_get_item_profile_nonexistent_item():
    # Préparer les données de test
    item_id = random.randint(364047, 99999999999)
    try:
        item_profile = get_item_profile(item_id, embeddings_df)
        assert False, "Failed: No exception raised for nonexistent item ID."
    except Exception as e:
        assert str(e) == f"Item ID {item_id} not found in embeddings dataframe.", "Failed: Incorrect exception message."
        
        
test_get_item_profile_existing_item()
test_get_item_profile_nonexistent_item()